In [1]:
js = {"annotation":[{"light_count":"1","box":[354,7,412,68],"attribute":[{"red":"off","green":"on","x_light":"off","others_arrow":"off","yellow":"off","left_arrow":"off"}],"type":"car","class":"traffic_light","direction":"vertical"},{"light_count":"1","box":[611,1,670,55],"attribute":[{"red":"off","green":"on","x_light":"off","others_arrow":"off","yellow":"off","left_arrow":"off"}],"type":"car","class":"traffic_light","direction":"horizontal"},{"light_count":"1","box":[858,14,919,65],"attribute":[{"red":"off","green":"on","x_light":"off","others_arrow":"off","yellow":"off","left_arrow":"off"}],"type":"car","class":"traffic_light","direction":"horizontal"}],"image":{"filename":"s01139934.jpg","imsize":[1280,720]}}
js

{'annotation': [{'light_count': '1',
   'box': [354, 7, 412, 68],
   'attribute': [{'red': 'off',
     'green': 'on',
     'x_light': 'off',
     'others_arrow': 'off',
     'yellow': 'off',
     'left_arrow': 'off'}],
   'type': 'car',
   'class': 'traffic_light',
   'direction': 'vertical'},
  {'light_count': '1',
   'box': [611, 1, 670, 55],
   'attribute': [{'red': 'off',
     'green': 'on',
     'x_light': 'off',
     'others_arrow': 'off',
     'yellow': 'off',
     'left_arrow': 'off'}],
   'type': 'car',
   'class': 'traffic_light',
   'direction': 'horizontal'},
  {'light_count': '1',
   'box': [858, 14, 919, 65],
   'attribute': [{'red': 'off',
     'green': 'on',
     'x_light': 'off',
     'others_arrow': 'off',
     'yellow': 'off',
     'left_arrow': 'off'}],
   'type': 'car',
   'class': 'traffic_light',
   'direction': 'horizontal'}],
 'image': {'filename': 's01139934.jpg', 'imsize': [1280, 720]}}

In [2]:
for d in js['annotation']:
    print(d)
    # x0,y0,x1,y1 = d["box"]  

{'light_count': '1', 'box': [354, 7, 412, 68], 'attribute': [{'red': 'off', 'green': 'on', 'x_light': 'off', 'others_arrow': 'off', 'yellow': 'off', 'left_arrow': 'off'}], 'type': 'car', 'class': 'traffic_light', 'direction': 'vertical'}
{'light_count': '1', 'box': [611, 1, 670, 55], 'attribute': [{'red': 'off', 'green': 'on', 'x_light': 'off', 'others_arrow': 'off', 'yellow': 'off', 'left_arrow': 'off'}], 'type': 'car', 'class': 'traffic_light', 'direction': 'horizontal'}
{'light_count': '1', 'box': [858, 14, 919, 65], 'attribute': [{'red': 'off', 'green': 'on', 'x_light': 'off', 'others_arrow': 'off', 'yellow': 'off', 'left_arrow': 'off'}], 'type': 'car', 'class': 'traffic_light', 'direction': 'horizontal'}


# 1) Setup

In [3]:
import os
import gc
import cv2
import numpy as np
import pandas as pd
from PIL import Image
from IPython.display import Image
from tqdm import tqdm
import json
import yaml
import shutil
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [4]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

Cloning into 'yolov5'...
remote: Enumerating objects: 15078, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 15078 (delta 0), reused 0 (delta 0), pack-reused 15075
Receiving objects: 100% (15078/15078), 14.12 MiB | 9.88 MiB/s, done.
Resolving deltas: 100% (10337/10337), done.
/content/yolov5
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 24.4 MB/s eta 0:00:00


In [5]:
import torch
from IPython.display import Image, clear_output
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

Setup complete. Using torch 1.13.1+cu116 (Tesla T4)


# 2) Load the data & Transfer COCO to Yolo format

### 데이터 업로드 (디렉토리 세팅 포함)

In [6]:
%cd ../

/content


In [7]:
# # 샘플데이터 업로드

# # 로컬에서 파일 업로드
# from google.colab import files
# uploaded = files.upload()

In [8]:
# os.mkdir('/content/sample_dataset')
# !unzip -qq '/content/sample.zip' -d '/content/yolov5/sample_dataset'

In [9]:
# 실제데이터 업로드

# Google Drive 에서 불러오기
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [10]:
os.mkdir('/content/yolov5/dataset')

In [34]:
!unzip -qq '/content/drive/MyDrive/Colab Notebooks/dataset_traffic.zip' -d '/content/yolov5/dataset'

replace /content/yolov5/dataset/images/train/s01544707.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/yolov5/dataset/images/train/s01544708.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace /content/yolov5/dataset/images/train/s01544708.jpg? [y]es, [n]o, [A]ll, [N]one, [r]ename: A


train, test 이미지 파일들의 경로 리스트를 txt 파일로 저장

In [35]:
from glob import glob

train_img_list = glob('/content/yolov5/dataset/images/train/*.jpg')
valid_img_list = glob('/content/yolov5/dataset/images/validation/*.jpg')
print(len(train_img_list),len(valid_img_list))

22177 7400


In [36]:
# txt 파일로 위 변수 리스트 저장

with open('/content/yolov5/dataset/train.txt', 'w') as f:
    f.write('\n'.join(train_img_list) + '\n')

with open('/content/yolov5/dataset/val.txt', 'w') as f:
    f.write('\n'.join(valid_img_list) + '\n')

### Json 파일을 Yolo Format으로 변환

In [37]:
json_path_list = glob('/content/yolov5/dataset/labels/*/*.json')
len(json_path_list) # json 파일 개수 확인

29577

In [15]:
# json 파일의 내용 형태 확인
# import pprint

# tmp = []
# for json_path in json_path_list[:2]:
#     print(json_path)
#     with open(json_path, "r") as jsondata:
#         json_data = json.load(jsondata)
#     tmp.append(json_data)
# pprint.pprint(tmp[0])
# pprint.pprint(tmp[1])

/content/yolov5/dataset/labels/validation/s01780640.json
/content/yolov5/dataset/labels/validation/s01801726.json
{'annotation': [{'box': [940, 194, 959, 215],
                 'class': 'traffic_sign',
                 'color': 'blue',
                 'kind': 'normal',
                 'shape': 'triangle',
                 'text': '0',
                 'type': 'instruction'},
                {'attribute': [{'green': 'on',
                                'left_arrow': 'off',
                                'others_arrow': 'off',
                                'red': 'off',
                                'x_light': 'off',
                                'yellow': 'off'}],
                 'box': [729, 152, 757, 160],
                 'class': 'traffic_light',
                 'direction': 'horizontal',
                 'light_count': '4',
                 'type': 'car'}],
 'image': {'filename': 's01780640.jpg', 'imsize': [1280, 720]}}
{'annotation': [{'attribute': [{'green': 'off',
  

In [17]:
def convert(size, box): #box: coco형식 xmin , ymin , w , h
    dw = 1/size[0]
    dh = 1/size[1]
    w = box[2]-box[0]
    h = box[3]-box[1]
    x = box[0]+ w/2
    y = box[1]+ h/2
    x = round(x*dw,6)
    w = round(w*dw,6)
    y = round(y*dh,6)
    h = round(h*dh,6)
    if w <0 or h < 0:
        return False
    return (x,y,w,h)

In [39]:
print(f'label 파일(json) 개수 : {len(json_path_list)}')
size = [1280,720]
class_names = []
for json_path in tqdm(json_path_list):
    with open(json_path, "r") as f:
        json_data = json.load(f)
    label_path = json_path.replace('.json','.txt')
    # img_path = json_path.replace('/labels','/images').replace('.json','.jpg') # 확인용
    # img = Image(img_path)
    # display(img)

    for d in json_data["annotation"]:
        box = d['box']
        bb = convert(size, box) # x0,y0,x1,y1 to x,y,w,h
        if bb==False:
            continue

        cls = d['class']
        if cls not in class_names:
            class_names.append(cls)

        line = f'{class_names.index(cls)} {bb[0]} {bb[1]} {bb[2]} {bb[3]}\n'
        with open(label_path, 'a') as f:
            f.write(line)
        
    shutil.move(json_path, '/content/yolov5/dataset/json_labels') # json 파일은 다른 폴더로 이동

label_path_list = glob("/content/yolov5/dataset/labels/*/*.txt")
print(f'변환후 label 파일(txt) 개수 : {len(label_path_list)}') # no objects in image, no *.txt file


label 파일(json) 개수 : 29577


  0%|          | 0/29577 [00:00<?, ?it/s]


FileNotFoundError: ignored

In [32]:
len(glob("/content/yolov5/dataset/labels/*/*.json"))

0

In [27]:
# 변환됐는지 확인
with open(label_path, "r") as f:
    while True:
        ln = f.readline()
        if not ln:
            break
        print(ln)

0 0.48125 0.179167 0.0625 0.077778

1 0.471875 0.294444 0.0125 0.005556

1 0.502734 0.290278 0.013281 0.005556



# 3) yaml 파일 만들기


In [28]:
# Create dataset.yaml file 
yaml_dict = {'train': 'content/yolov5/dataset/train.txt',   # path to the train folder
            'val': 'content/yolov5/dataset/val.txt', # path to the val folder
            'nc': 2,                             # number of classes
            'names': ['traffic_sign', 'traffic_light']}         # list of label names

# we will make the file under the yolov5/data/ directory.
data_yaml = '/content/yolov5/dataset/dataset.yaml'
with open(data_yaml, 'w') as f:
    yaml.dump(yaml_dict, f, default_flow_style=True)

In [29]:
%cat '/content/yolov5/dataset/dataset.yaml' # show your YAML file

{names: [traffic_sign, traffic_light], nc: 2, train: content/yolov5/dataset/train.txt,
  val: content/yolov5/dataset/val.txt}


In [18]:
# # 다른 방법

# # 사용할 데이터셋 경로 및 데이터셋의 yaml 파일 경로 지정
# data_dir = '/content/dataset'
# data_yaml = '/content/dataset/dataset.yaml'

# # 데이터셋 yaml 파일 확인 
# with open(data_yaml) as f:
#     film = yaml.load(f, Loader=yaml.FullLoader)
#     display(film)

In [ ]:
# # yaml 파일의 train, val 데이터가 있는 경로 수정 (기존 경로 -> 구글 드라이브에 저장된 경로로)
# film['train'] = '/content/dataset/train/images'
# film['val'] = '/content/dataset/validation/images'

# with open(data_yaml, 'w') as f:
#     yaml.dump(film, f)

# print('변경된 yaml 파일 :')
# with open(data_yaml) as f:
#     film = yaml.load(f, Loader=yaml.FullLoader)
#     display(film)

# 4) 모델 구성

# 5) 모델 학습

Weights and Biases Logging

In [ ]:
%load_ext tensorboard
%tensorboard --logdir /kaggle/training/yolov5/runs

In [ ]:
# Install W&B 
%pip install -q --upgrade wandb
# Login with token, follow with the guide
import wandb
wandb.login()

In [2]:
from glob import glob